## Building A Chatbot
implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation.

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions



In [11]:
#pip install --upgrade pydantic
pip uninstall pydantic



SyntaxError: invalid syntax. Perhaps you forgot a comma? (3677944788.py, line 2)

In [13]:
pip show pydantic


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F9094DA6E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F9094D95D0>, model_name='Gemma2-9b-It')

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Abhishek and I am a Chief AI Engineer")])

AIMessage(content="Hello Abhishek, it's a pleasure to meet you!\n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 22, 'total_tokens': 76, 'completion_time': 0.098181818, 'prompt_time': 0.000139959, 'queue_time': 0.002761491, 'total_time': 0.098321777}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8299d400-5944-4114-9140-1c2cd25de193-0', usage_metadata={'input_tokens': 22, 'output_tokens': 54, 'total_tokens': 76})

In [6]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Abhishek and I am a Chief AI Engineer"),
        AIMessage(content="Hello Abhishek! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)
# AI message is copied from the reponse before this

AIMessage(content="According to our conversation, your name is Abhishek and you are a Chief AI Engineer. 😊  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm interested in learning more! \n\n", response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 97, 'total_tokens': 148, 'completion_time': 0.092727273, 'prompt_time': 0.002531962, 'queue_time': 0.0038254779999999998, 'total_time': 0.095259235}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3d3a9d67-9837-4694-998f-c8cf7ec7dce6-0', usage_metadata={'input_tokens': 97, 'output_tokens': 51, 'total_tokens': 148})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [7]:
!pip install langchain_community

In [17]:
from langchain_community.chat_message_histories import ChatMessageHistory
# 
from langchain_core.chat_history import BaseChatMessageHistory
#
from langchain_core.runnables.history import RunnableWithMessageHistory
#

store={}

# make different sessions for different users
# return type for below is BaseChatMessageHistory
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
        #ChatMessageHistory is a class which i use above
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)
#read about the RunnableWithMessageHistory

In [18]:
config={"configurable":{"session_id":"chat1"}}

In [19]:
# i am using the above session id here
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Abhishek and I am a Chief AI Engineer")],
    config=config
)

In [20]:
response.content

"Hello Abhishek,\n\nIt's a pleasure to meet you. Being a Chief AI Engineer is a fascinating role! \n\nWhat are you currently working on that you find most exciting?  I'm always eager to learn about the latest advancements in AI.\n"

In [21]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Abhishek.  \n\nWe established that at the beginning of our conversation. 😊  Is there something else I can help you with? \n', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 91, 'total_tokens': 124, 'completion_time': 0.06, 'prompt_time': 0.003504267, 'queue_time': 0.06938265199999999, 'total_time': 0.063504267}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9307d2a2-aa85-4541-b71e-66f74ccfecc9-0', usage_metadata={'input_tokens': 91, 'output_tokens': 33, 'total_tokens': 124})

In [22]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it! 😊\n"

In [23]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Naveen")],
    config=config1
)
response.content

"Hi Naveen, it's nice to meet you! 👋  \n\nHow can I help you today?\n"

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Naveen! 😊  \n\nI remember you telling me.  \n\nIs there anything else I can help you with today?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. First, let's add in a system message with some custom instructions (but still taking messages as input).

In [62]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [63]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Abhishek")]})

AIMessage(content="Hello Abhishek, it's nice to meet you! \n\nI'm ready to help with any questions you have.  Ask away!  😊 \n", response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 31, 'total_tokens': 66, 'completion_time': 0.063636364, 'prompt_time': 0.001620045, 'queue_time': 0.0029929539999999995, 'total_time': 0.065256409}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8840c789-b690-405f-bdc4-376175ccede9-0', usage_metadata={'input_tokens': 31, 'output_tokens': 35, 'total_tokens': 66})

In [64]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [65]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Rahul")],
    config=config
)

response

AIMessage(content="Hi Rahul!  It's nice to meet you. 😊  \n\nLet me know if you have any questions or if there's anything I can help you with. \n", response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 281, 'total_tokens': 320, 'completion_time': 0.070909091, 'prompt_time': 0.009189498, 'queue_time': 0.0038627819999999986, 'total_time': 0.080098589}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c59be797-19ba-43cd-8fd7-ebb22f6f4c54-0', usage_metadata={'input_tokens': 281, 'output_tokens': 39, 'total_tokens': 320})

In [66]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"Your name is Rahul!  \n\nI'm learning new things all the time, including names. 😄  How can I help you today? \n"

In [67]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [68]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Abhishek")],"language":"Hindi"})
response.content

'नमस्ते अभिषेक! \n\nमुझे आपसे मिलकर खुशी हो रही है। क्या मैं आपकी कोई मदद कर सकता हूँ?  \n'

This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [70]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [71]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Abhishek")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते अभिषेक! 😊 \n\nआप कैसे हैं?  \n'

In [54]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [72]:
response.content

'नमस्ते अभिषेक! \n\nमुझे आपसे मिलकर खुशी हो रही है। क्या मैं आपकी कोई मदद कर सकता हूँ?  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [37]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
# last is last conversation
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"